<a href="https://colab.research.google.com/github/hanghae-plus-AI/AI-1-hyein0623/blob/main/chat4_%EC%8B%AC%ED%99%94%EA%B3%BC%EC%A0%9C_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace로 영화리뷰 감정 분석 모델 구현하기

In [ ]:
!pip install transformers datasets evaluate accelerate scikit-learn

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00


In [ ]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Dataset 준비

MNLI 데이터 준비

In [ ]:
mndb = load_dataset("nyu-mll/glue", "mnli")
mndb

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [ ]:
mndb['train'][0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(data):
    return tokenizer(data["premise"], data["hypothesis"],truncation=True, max_length = 128)

mndb_tokenized = mndb.map(preprocess_function, batched=True)

# train, val, test 데이터 지정
mndb_split = mndb_tokenized['train'].train_test_split(test_size=0.2)
mndb_train, mndb_val = mndb_split['train'], mndb_split['test']
mndb_test = mndb_tokenized['validation_matched']

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

Tokenizer를 실행할 때 넘겨주었던 `truncation` 옵션은 주어진 text가 일정 길이 이상이면 잘라내라는 의미입니다.
만약 특정 길이 값이 같이 주어지지 않는다면 `bert-base-cased`를 학습할 때 사용한 text의 최대 길이를 기준으로 값을 결정합니다.

In [ ]:
mndb_train[0].keys()

dict_keys(['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
len(mndb_train), len(mndb_val), len(mndb_test)

(314161, 78541, 9815)

In [ ]:
# 라벨확인
label_names =mndb_train.features['label'].names

for idx in range(len(label_names)):
  num = mndb_train['label'].count(idx)
  print(f"{label_names[idx]} : {num}")



entailment : 104772
neutral : 104685
contradiction : 104704


In [ ]:
class_num = len(label_names)
class_num

3

## Model 구현

이번에는 text 분류를 수행할 Transformer를 구현합니다.
이전에는 Transformer의 구성 요소들을 직접 구현하여 합쳤습니다.
이번에는 HuggingFace의 BERT를 활용하여 인자만 넘겨주는 식으로 구현해보겠습니다:

In [ ]:
from transformers import BertConfig

config = BertConfig()

config.hidden_size = 64  # BERT layer의 기본 hidden dimension
config.intermediate_size = 64  # FFN layer의 중간 hidden dimension
config.num_hidden_layers = 2  # BERT layer의 개수
config.num_attention_heads = 4  # Multi-head attention에서 사용하는 head 개수
config.num_labels = class_num  # 마지막에 예측해야 하는 분류 문제의 class 개수

model = AutoModelForSequenceClassification.from_config(config)

## 학습 코드

다음은 위에서 구현한 Transformer를 imdb로 학습하는 코드를 구현합니다.
먼저 다음과 같이 학습 인자들을 정의합니다.

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=10,  # epoch 수
    per_device_train_batch_size=128,  # training data의 batch size
    per_device_eval_batch_size=128,  # validation data의 batch size
    logging_strategy="epoch",  # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-3,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")


def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import EarlyStoppingCallback


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mndb_train,
    eval_dataset=mndb_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    # callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

모델, training 인자, training과 validation data, 부가적인 평가 함수, 그리고 tokenizer를 넘겨주면 끝입니다.
별개로 early stopping과 같은 기능도 주석 친 부분과 같이 `callbacks`로 구현할 수 있으니 참고해주시길 바랍니다.

위와 같이 만든 `Trainer`는 다음과 같이 학습을 할 수 있습니다.

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.969800,0.915962,0.559313
2,0.870900,0.867510,0.604767
3,0.814400,0.853214,0.613590
4,0.766800,0.865451,0.614214
5,0.726200,0.885892,0.616162
6,0.688900,0.917469,0.615398
7,0.655800,0.937410,0.610344
8,0.623200,0.981695,0.605709
9,0.594800,1.012341,0.602984
10,0.573800,1.051651,0.601495


TrainOutput(global_step=24550, training_loss=0.7284666128993763, metrics={'train_runtime': 1141.4162, 'train_samples_per_second': 2752.379, 'train_steps_per_second': 21.508, 'total_flos': 116464307399370.0, 'train_loss': 0.7284666128993763, 'epoch': 10.0})

In [ ]:
trainer.evaluate(mndb_test)

{'eval_loss': 0.8343952298164368,
 'eval_accuracy': 0.6266938359653591,
 'eval_runtime': 1.9167,
 'eval_samples_per_second': 5120.685,
 'eval_steps_per_second': 40.172,
 'epoch': 10.0}

이전에 학습 인자에서 `load_best_model_at_end=True`를 넘겨줬기 때문에 `trainer`는 학습이 끝난 후, 기본적으로 validation loss가 가장 좋은 모델을 가지고 `evaluate`를 진행합니다.
실제로 결과를 보면 `eval_loss`가 가장 낮은 validation loss와 유사한 것을 볼 수 있습니다.

평가할 때 사용한 모델은 다음과 같이 저장할 수 있습니다.

In [ ]:
trainer.save_model()

그리고 저장한 모델을 가지고 다른 예시들을 예측하는 것은 다음과 같이 구현할 수 있습니다.

In [ ]:
from transformers import pipeline


classifier = pipeline("sentiment-analysis", model="./hf_transformer/", device='cuda')
# print(classifier("The movie was so disgusting..."))
# print(classifier("The movie was so amazing!!"))

# overfiting  방지


* overfiting 방지하기 위해
  - earlyStopping 추가
  - 1e−4 : 기존의 학습률(LR) 변경 1e-3  >>  1e-4

* 모델의 복잡도를 올림  
  - 입력 데이터의 두문장의 논리 관계를 포함해야한다고 판단하는 task로 데이터가 복잡하다고 생각
  - 모델의 복잡도를 올려줌
  

In [ ]:
from transformers import BertConfig

config = BertConfig()

config.hidden_size = 128 # BERT layer의 기본 hidden dimension
config.intermediate_size = 128  # FFN layer의 중간 hidden dimension
config.num_hidden_layers = 2  # BERT layer의 개수
config.num_attention_heads = 4  # Multi-head attention에서 사용하는 head 개수
config.num_labels = class_num  # 마지막에 예측해야 하는 분류 문제의 class 개수

model = AutoModelForSequenceClassification.from_config(config)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments (
    output_dir='hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=20,  # epoch 수
    per_device_train_batch_size=128,  # training data의 batch size
    per_device_eval_batch_size=128,  # validation data의 batch size
    logging_strategy="epoch",  # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-4,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

In [ ]:
from transformers import EarlyStoppingCallback


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mndb_train,
    eval_dataset=mndb_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.006800,0.954431,0.536420
2,0.929500,0.939902,0.548949
3,0.878500,0.902111,0.570441
4,0.836700,0.909147,0.576132


TrainOutput(global_step=9820, training_loss=0.9128482608834012, metrics={'train_runtime': 485.6883, 'train_samples_per_second': 12936.733, 'train_steps_per_second': 101.094, 'total_flos': 183573713065068.0, 'train_loss': 0.9128482608834012, 'epoch': 4.0})

In [ ]:
from transformers import BertConfig

config = BertConfig()

config.hidden_size = 32 # BERT layer의 기본 hidden dimension
config.intermediate_size = 32  # FFN layer의 중간 hidden dimension
config.num_hidden_layers = 2  # BERT layer의 개수
config.num_attention_heads = 4  # Multi-head attention에서 사용하는 head 개수
config.num_labels = class_num  # 마지막에 예측해야 하는 분류 문제의 class 개수

model = AutoModelForSequenceClassification.from_config(config)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments (
    output_dir='hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=40,  # epoch 수
    per_device_train_batch_size=128,  # training data의 batch size
    per_device_eval_batch_size=128,  # validation data의 batch size
    logging_strategy="epoch",  # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-4,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

In [ ]:
from transformers import EarlyStoppingCallback


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mndb_train,
    eval_dataset=mndb_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.902600,0.963641,0.534702
2,0.886600,0.973041,0.535033


TrainOutput(global_step=4910, training_loss=0.8945683434635947, metrics={'train_runtime': 222.2785, 'train_samples_per_second': 56534.665, 'train_steps_per_second': 441.788, 'total_flos': 6005207544630.0, 'train_loss': 0.8945683434635947, 'epoch': 2.0})

In [ ]:
trainer.evaluate(mndb_test)

{'eval_loss': 0.9554723501205444,
 'eval_accuracy': 0.5451859398879266,
 'eval_runtime': 3.7093,
 'eval_samples_per_second': 2646.072,
 'eval_steps_per_second': 20.759,
 'epoch': 2.0}

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline


classifier = pipeline("sentiment-analysis", model="./hf_transformer/", device='cuda')
# print(classifier("The movie was so disgusting..."))
# print(classifier("The movie was so amazing!!"))